In [22]:
import boto3
import configparser
import time
import glob
import os
import pandas as pd
import json
import pyarrow.parquet as pq

# Check data ny taxi

In [51]:
path_data = r'D:\DA\prj_pipeline\ny_taxi\yellow_tripdata_2024-01.parquet'
# check metadata
print("metadata")
print(pq.read_metadata(path_data))

metadata
  created_by: parquet-cpp-arrow version 14.0.2
  num_columns: 19
  num_rows: 2964624
  num_row_groups: 3
  format_version: 2.6
  serialized_size: 6357


In [62]:
file = pq.ParquetFile(path_data)
table = file.read()
table.schema

VendorID: int32
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: int64
trip_distance: double
RatecodeID: int64
store_and_fwd_flag: large_string
PULocationID: int32
DOLocationID: int32
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
Airport_fee: double

In [63]:
batches_iter = file.iter_batches(batch_size=10)
df = next(batches_iter).to_pandas()
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.00
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.00
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.00
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.00
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.00
5,1,2024-01-01 00:54:08,2024-01-01 01:26:31,1,4.70,1,N,148,141,1,29.6,3.5,0.5,6.90,0.0,1.0,41.50,2.5,0.00
6,2,2024-01-01 00:49:44,2024-01-01 01:15:47,2,10.82,1,N,138,181,1,45.7,6.0,0.5,10.00,0.0,1.0,64.95,0.0,1.75
7,1,2024-01-01 00:30:40,2024-01-01 00:58:40,0,3.00,1,N,246,231,2,25.4,3.5,0.5,0.00,0.0,1.0,30.40,2.5,0.00
8,2,2024-01-01 00:26:01,2024-01-01 00:54:12,1,5.44,1,N,161,261,2,31.0,1.0,0.5,0.00,0.0,1.0,36.00,2.5,0.00
9,2,2024-01-01 00:28:08,2024-01-01 00:29:16,1,0.04,1,N,113,113,2,3.0,1.0,0.5,0.00,0.0,1.0,8.00,2.5,0.00


In [64]:
df.to_csv('test.csv', index=True)

In [73]:

data =r'D:\DA\prj_pipeline\ny_taxi\taxi_zone_lookup.csv'
# Setting the id column as the index
airbnb_data = pd.read_csv(data)
# airbnb_data = pd.read_csv("data/listings_austing.csv", index_col=0)

# Preview first 5 rows
airbnb_data.head(50)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone


# Loading all Params from dwh.cfh file

In [27]:

config = configparser.ConfigParser()
config.read('dwh.cfg')
KEY = config['AWS']['KEY']
SECRET = config['AWS']['SECRET']
REGION_NAME = config['AWS']['REGION_NAME']

DWH_CLUSTER_TYPE = config['DWH']['DWH_CLUSTER_TYPE']
DWH_NUM_NODES = config['DWH']['DWH_NUM_NODES']
DWH_NODE_TYPE = config['DWH']['DWH_NODE_TYPE']
DWH_IAM_ROLE_NAME = config['DWH']['DWH_IAM_ROLE_NAME']
DWH_CLUSTER_IDENTIFIER = config['DWH']['DWH_CLUSTER_IDENTIFIER']
DWH_DB = config['DWH']['DWH_DB']
DWH_DB_USER = config['DWH']['DWH_DB_USER']
DWH_DB_PASSWORD = config['DWH']['DWH_DB_PASSWORD']
DWH_PORT = config['DWH']['DWH_PORT']

pd.DataFrame({"Param":
              ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
            [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
})

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,t2.micro
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,test_iam_role


# Creating clients for EC2, IAM, redshift

In [45]:
# create IAM client
iam  = boto3.client('iam',aws_access_key_id = KEY, aws_secret_access_key = SECRET, region_name = REGION_NAME )

# create s3 client
s3  = boto3.client('s3',aws_access_key_id = KEY, aws_secret_access_key = SECRET, region_name = REGION_NAME )
s3_rsc  = boto3.resource('s3',aws_access_key_id = KEY, aws_secret_access_key = SECRET, region_name = REGION_NAME )

#create EC2 client
ec2  = boto3.client('ec2',aws_access_key_id = KEY, aws_secret_access_key = SECRET, region_name = REGION_NAME )

# create redshift client
redshift  = boto3.client('redshift',aws_access_key_id = KEY, aws_secret_access_key = SECRET, region_name = REGION_NAME )


In [29]:
try:
  print('create iam role')
  dwhRole = iam.create_role(Path= '/', RoleName = DWH_IAM_ROLE_NAME,
                            Description = 'Allows Redshift clusters to call AWS services on your behalf.',
                            AssumeRolePolicyDocument = json.dumps(
                              {'Statement' : [{'Action': 'sts:AssumeRole',
                                    'Effect': 'Allow',
                                    'Principal': {'Service': 'redshift.amazonaws.com'}}],
                                 'Version': '2012-10-17'})
                            )
except Exception as e:
  print(e)

print("Attaching Policy")

try:
    iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess"
                      )['ResponseMetadata']['HTTPStatusCode']
    print("Get the IAM role ARN")
    roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
    print(roleArn)
except Exception as e:
    print(e)

create iam role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name test_iam_role already exists.
Attaching Policy
Get the IAM role ARN
arn:aws:iam::590183675741:role/test_iam_role


# create s3 bucket unprocessed 

In [47]:
# def bucket_s3_exists(s3_rsc, b):
#   s3_rsc = boto3.resource('s3')
#   return s3_rsc.Bucket(b) in s3_rsc.buckets.all()

def create_s3_bucket(s3, b, folders):
  s3.create_bucket(Bucket = b, CreateBucketConfiguration={'LocationConstraint': 'ap-southeast-2'})
  if folders !=  ' ':
    fls = folders.split(',')
    for f in fls:
      s3.put_object(Bucket = b, Body = ' ', Key = f + '/')

tt = time.time_ns()
prefix = f'ny-taxi-bucket-s3-{tt}'
bucket_names = {prefix: 'unprocessed_reports'}

# creating s3 buckets.....................................
for k in bucket_names:
  create_s3_bucket(s3, k, bucket_names[k])

print('S3 buckets created')

S3 buckets created


# upload file to s3 

In [48]:
def upload_files_to_s3(s3, file_name, b, folder, object_name, args=None):
  if object_name is None:
    object_name = folder + "/{fname}".format(fname = os.path.basename(file_name))

  response = s3.upload_file(file_name, b, object_name, ExtraArgs=None)
  return response

print('Uploading the local receipts files to ny-taxi-bciket-s3 AWS S3 bucket...')
file = 'test.csv'
print('upload_file:', file)
print(upload_files_to_s3(s3, file, prefix, 'unprocessed_reports', None))

print(f'Files uploaded to {prefix} AWS S3 bucket')
print(f'ID: {tt}')

Uploading the local receipts files to ny-taxi-bciket-s3 AWS S3 bucket...
upload_file: test.csv
None
Files uploaded to ny-taxi-bucket-s3-1729690152438445900 AWS S3 bucket
ID: 1729690152438445900


# Creating redshift cluster

In [ ]:
try:
  response = redshift.create_cluster(
    ClusterType = DWH_CLUSTER_TYPE,
    NodeType = DWH_NODE_TYPE,
    NumberOfNodes = int(DWH_NUM_NODES),

# <-------------------------------------------------------------->

  DBName = DWH_DB,
  ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
  MasterUsername = DWH_DB_USER,
  MasterUserPassword = DWH_DB_PASSWORD,


  IamRoles = [roleArn]
  )
except Exception as e:
  print(e)